In [4]:
#library
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
# The number of times the word is repeated
foods= pd.read_csv('food11.csv')
import nltk   #natural language toolkit

vocabulary= nltk.FreqDist()

for ingredients in foods['ingredients']:
    ingredients = ingredients.split("|")
    vocabulary.update(ingredients)
for word, frequency in vocabulary.most_common(300):       #300 ta az portekrar tarin kalame ro neshon bd
    print(f'{word};{frequency}')


نمک;766
فلفل سیاه;601
پیاز;515
آب;498
روغن مایع;398
زردچوبه;396
سیر;182
تخم مرغ;172
آرد سفید;164
روغن مایع معمولی;162
کره;157
شکر;154
شیرگاو;148
برنج;139
رب گوجه فرنگی;118
زعفران;111
دارچین;106
گوجه فرنگی;90
نعنا خشک;89
گردو;84
گوشت چرخ کرده;80
سیب زمینی;80
خامه;79
نخود;69
جعفری;67
هل;66
آبلیمو;65
گوشت گوساله;65
گوشت سینه مرغ;63
ماست;62
آویشن;60
روغن زیتون;59
عدس;59
قارچ;52
فلفل دلمه ای;48
زیره سیاه;48
فلفل قرمز;47
کشمش;47
بادمجان;46
هویج;45
گوشت گوسفند;45
گوشت مرغ;43
لپه;42
گشنیز;41
رب انار;40
گلاب;39
سبزی آش;39
بکینگ پودر;39
وانیل;36
لوبیا چیتی;36
پنیر موزارِلّا (MOZZARELLA);34
تخم گشنیز;34
بلغور گندم;33
خلال بادام;31
پاپریکا;30
آرد سوخاری;30
شوید;28
سس مایونز;27
پیازچه;27
عسل;27
آبغوره;27
کشک;27
لیمو ترش;26
لوبیا قرمز;25
آرد نخودچی;24
کنجد;23
لیمو عُمانی (اَمانی);23
ماش;22
پودر زنجبیل;21
جوز هندی;21
لوبیا سفید;20
برگ بو;20
چای سیاه;20
میخک;20
پودر سیر;19
نشاسته ذرت;19
سبزی مخصوص;19
فیله ماهی;19
اسفناج;19
خرما;18
سرکه;18
ذرت;18
پودر نارگیل;17
فلفل سبز;17
کرفس;17
نعناع;17
آب نارنج;16


In [6]:
def load_foods_data():
    # Load DataFrame
    data1 = pd.read_csv('food11.csv', index_col='recipe_id')
    return data1
def suggest_recipes_with_preferences(desired_ingredients, allergic_ingredients, num_suggestions=10):
    # Load foods data
    foods = load_foods_data()

    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform ingredients
    ingredient_vectors = vectorizer.fit_transform(foods['ingredients'])

    # Transform desired ingredients into a vector
    desired_ingredients_vector = vectorizer.transform(desired_ingredients)

    # Transform allergic ingredients into a vector
    allergic_ingredients_vector = vectorizer.transform(allergic_ingredients)

    # Calculate similarity between desired ingredients and all recipes
    desired_similarity_scores = cosine_similarity(desired_ingredients_vector, ingredient_vectors).flatten()

    # Calculate similarity between allergic ingredients and all recipes
    allergic_similarity_scores = cosine_similarity(allergic_ingredients_vector, ingredient_vectors).flatten()

    # Combine the similarity scores, giving higher weight to desired ingredients and lower weight to allergic ingredients
    combined_similarity_scores = desired_similarity_scores - allergic_similarity_scores

    # Get indices of top similarity scores
    top_indices = combined_similarity_scores.argsort()[::-1][:num_suggestions]

    # Get top recipe suggestions
    suggested_recipes = foods.iloc[top_indices]

    # Sort recipes by combined similarity score in descending order
    suggested_recipes['combined_similarity_score'] = combined_similarity_scores[top_indices]
    suggested_recipes.sort_values(by='combined_similarity_score', ascending=False, inplace=True)

    return suggested_recipes
desired_ingredients = input("Enter desired ingredients (separated by commas): ")
desired_ingredients = desired_ingredients.lower().split(",")

allergic_ingredients = input("Enter allergic ingredients (separated by commas): ")
allergic_ingredients = allergic_ingredients.lower().split(",")

suggested_recipes = suggest_recipes_with_preferences(desired_ingredients, allergic_ingredients)
test = pd.DataFrame(suggested_recipes)
test

Enter desired ingredients (separated by commas): پیاز
Enter allergic ingredients (separated by commas): مرغ


<ipython-input-6-85049892c774>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suggested_recipes['combined_similarity_score'] = combined_similarity_scores[top_indices]
<ipython-input-6-85049892c774>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suggested_recipes.sort_values(by='combined_similarity_score', ascending=False, inplace=True)


,title,categories,ingredients,calories,combined_similarity_score
recipe_id,,,,,
1089,آش ترخینه کرمانشاه,آش,پیاز|پیاز|نمک|زردچوبه|روغن مایع|آب|ترخینه,702.5,0.419330
1095,پیاز داغ,دستورات مکمل,پیاز|نمک|زردچوبه|روغن مایع معمولی|آب,6155.0,0.344873
973,قیمه ریزه سیب زمینی,کوفته_کبه,پیاز|نمک|فلفل سیاه|روغن مایع|گوشت چرخ کرده|پیا...,23171.5,0.320802
1004,شفته انار,کوفته_کبه,پیاز|نمک|فلفل سیاه|زردچوبه|روغن مایع|گوشت چرخ ...,8176.0,0.301993
484,خورش فسنجان گردو با گوشت قلقلی,خورشت,پیاز|روغن مایع|رب انار|شکر|زعفران|گردو|پیاز|نم...,10449.5,0.282464
1003,شفته سماق,کوفته_کبه,پیاز|نمک|فلفل سیاه|زردچوبه|روغن مایع|گوشت چرخ ...,10643.5,0.279386
480,خورش فسنجان بادام با گوشت,خورشت,پیاز|روغن مایع|رب انار|شکر|زعفران|بادام|پیاز|ن...,5661.0,0.278065
482,خورش فسنجان فندق با گوشت قلقلی,خورشت,پیاز|روغن مایع|رب انار|شکر|زعفران|فندق|پیاز|نم...,11327.5,0.263684
477,خورش داوود پاشا,خورشت,پیاز|آرد سفید|نمک|رب انار|نعنا خشک|پیاز|نمک|فل...,34766.5,0.262812


In [7]:
ingredients = test.loc[1089]['ingredients'].split('|')
print(ingredients)

['پیاز', 'پیاز', 'نمک', 'زردچوبه', 'روغن مایع', 'آب', 'ترخینه']
